<a href="https://colab.research.google.com/github/carloshcamp/DSWP-CURSO-OUT-NOV-2020/blob/main/Exerc%C3%ADcio_Kaggle_aula_35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

     |████████████████████████████████| 256kB 4.6MB/s 
     |████████████████████████████████| 148.9MB 71kB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 276kB 40.9MB/s 
     |████████████████████████████████| 66.3MB 145kB/s 
     |████████████████████████████████| 1.8MB 39.2MB/s 
     |████████████████████████████████| 174kB 43.8MB/s 
     |████████████████████████████████| 13.9MB 39.1MB/s 
     |████████████████████████████████| 6.8MB 44.1MB/s 
     |████████████████████████████████| 1.6MB 31.6MB/s 
     |████████████████████████████████| 266kB 39.1MB/s 
     |████████████████████████████████| 2.1MB 41.7MB/s 
     |████████████████████████████████| 337kB 41.1MB/s 
     |████████████████████████████████| 1.1MB 36.4MB/s 
     |████████████████████████████████| 153kB 39.8MB/s 
     |████████████████████████████████| 163kB 48.3MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 61kB 6.3M

In [2]:
!pip install shap

     |████████████████████████████████| 327kB 5.9MB/s 
  Created wheel for shap: filename=shap-0.37.0-cp36-cp36m-linux_x86_64.whl size=463905 sha256=c09acf3729df6db4ba237087ba213fd05fae2e0012e980d624dd93e4e57f79fd
  Stored in directory: /root/.cache/pip/wheels/df/ad/b0/aa7815ec68850d66551ef618095eccb962c8f6022f1d3dd989
Successfully built shap


In [5]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as pyplot
import seaborn as sns

sns.set_theme(style = 'ticks', palette = 'pastel')

from pycaret import regression, classification
import shap

from collections import Counter

import tensorflow as tf
from tensorflow import keras


In [6]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [7]:
pd.set_option('display.precision',3)

In [21]:
pd.options.display.max_columns = 50

In [22]:
pd.options.display.max_rows = 50

In [12]:
from IPython.core.display import display, HTML
import base64

In [13]:
!pip install facets-overview==1.0.0


In [15]:
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

In [16]:
url_train = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/train_LABDATA.csv'
url_test = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/test_LABDATA.csv'

In [38]:
df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)

In [39]:
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [40]:
df_train.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [41]:
df_train.dtypes

id                    int64
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [42]:
df_train['TotalCharges'] = df_train['TotalCharges'].str.strip().replace('',np.nan)

In [43]:
df_train['TotalCharges'].value_counts()

20.2      8
19.55     7
20.05     6
19.45     6
20.25     6
         ..
813.45    1
1445.3    1
573.15    1
54.35     1
90.6      1
Name: TotalCharges, Length: 5285, dtype: int64

In [44]:
df_train.isna().sum()

id                    0
gender                0
SeniorCitizen         0
Partner               0
Dependents          218
tenure              461
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod        99
MonthlyCharges        0
TotalCharges          8
Churn                 0
dtype: int64

In [45]:
df_train['TotalCharges_2'] = df_train['TotalCharges'].astype('float64')
#df_train['TotalCharges_2'] = pd.to_numeric(df_train['TotalCharges'], errors = 'coerce')
#df_treino[df_treino['TotalCharges'].str.contains('[^\d\.]', regex=True)]

In [47]:
df_train = df_train.drop(columns=['TotalCharges'])

In [46]:
df_train.isna().sum()

id                    0
gender                0
SeniorCitizen         0
Partner               0
Dependents          218
tenure              461
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod        99
MonthlyCharges        0
TotalCharges          8
Churn                 0
TotalCharges_2        8
dtype: int64

In [49]:
l_features_num = list(df_train.select_dtypes([np.float64,np.int64]))

In [50]:
l_features_num

['id', 'SeniorCitizen', 'tenure', 'MonthlyCharges', 'Churn', 'TotalCharges_2']

In [59]:
df_train['SeniorCitizen_2']=df_train['SeniorCitizen'].replace({0:'Não',1:'Sim'})

In [60]:
df_train = df_train.drop(columns=['SeniorCitizen'])

In [61]:
Counter(df_train['SeniorCitizen_2'])

Counter({'Não': 4740, 'Sim': 894})

In [53]:
df_train.dtypes

id                     int64
gender                object
SeniorCitizen       category
Partner               object
Dependents            object
tenure               float64
PhoneService          object
MultipleLines         object
InternetService       object
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract              object
PaperlessBilling      object
PaymentMethod         object
MonthlyCharges       float64
Churn                  int64
TotalCharges_2       float64
dtype: object

In [55]:
l_features_char = list(df_train.select_dtypes([np.object]))

In [57]:
def mostra_dados(df):
  fsg = FeatureStatisticsGenerator()
  dataframes = [
      {'table': df, 'name': 'Data'}]

  censusProto = fsg.ProtoFromDataFrames(dataframes)
  protostr = base64.b64encode(censusProto.SerializeToString()).decode("utf-8")

  HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
  html = HTML_TEMPLATE.format(protostr=protostr)
  display(HTML(html))

In [62]:
mostra_dados(df_train)